## JuTrack Integration

In [ ]:
import numpy as np
import pandas as pd
from scipy.constants import c, e, physical_constants

from synapticTrack.beam import *
from synapticTrack.io import *
from synapticTrack.visualizations import *

## Compute Beam Center

### Track Input Beam Distribution

In [ ]:
base_dir = '../data/input_beam'
filename = base_dir + '/' + 'coord.out'

beam_io_manager = BeamDataIOManager() 
beam = beam_io_manager.read(code='track', filename=filename, mass_number=40, charge_state=8, beam_current=0, reference_energy=0.010)
beam.state

In [ ]:
x  = beam.x
xp = beam.xp
y  = beam.y
yp = beam.yp
dt = beam.dt
dW = beam.dW

In [ ]:
beam.centroid

In [ ]:
print(beam.charge_state, beam.mass_number, beam.reference_energy)

In [ ]:
twiss = Twiss(beam)
print(twiss.values()['twiss_x']['emittance'])
print(twiss.values()['twiss_y']['emittance'])
print(twiss.values()['twiss_z']['emittance'])

## Track to JuTrack Conversion of Input Beam Distribution

### Test

In [ ]:
beam_state = pd.DataFrame({
    'x': [1.0],     # mm
    'xp': [10.0],   # mrad
    'y': [2.0],     # mm
    'yp': [-5.0],   # mrad
    'dt': [1e-9],   # sec
    'dW': [0.001]   # MeV/u
})

In [ ]:
beam_state

In [ ]:
A = 40         # mass number
Z = 8          # charge state
I = 0          # beam current
Ek = 0.01      # kinetic energy in MeV/u
test_beam = Beam(beam_state, A, Z, I, Ek)
jutrack_coords = JuTrackIO.convert_to_jutrack_coordinates(test_beam)
print(jutrack_coords)

### Track to JuTrack Conversion of Input Beam Distribution

In [ ]:
jutrack_coords = JuTrackIO.convert_to_jutrack_coordinates(beam)
jutrack_coords

In [ ]:
xjt = jutrack_coords['x']
xpjt = jutrack_coords['px_p0']
yjt = jutrack_coords['y']
ypjt = jutrack_coords['py_p0']
z = jutrack_coords['z']
delta = jutrack_coords['delta']

In [ ]:
x0 = beam.centroid['x']
xp0 = beam.centroid['xp']
y0 = beam.centroid['y']
yp0 = beam.centroid['yp']

### Phasespace Plots of Initial Beam Distribution with Jutrack

In [ ]:
phasespace_plot(xjt*1e3, xpjt*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -20, 20], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x/p_0$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
phasespace_plot(yjt*1e3, ypjt*1e3, x_center=y0, y_center=yp0, xyrange=[-20, 20, -20, 20], title='Vertical Phasespace Distribution', 
                xlabel=r'$y$ [mm]', ylabel=r"$y^{\prime}$ [mrad]",
                nbins=100, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
phasespace_plot(z*1e-6, delta*1e-2, xyrange=[-20, 20, -10e-6, 10e-6], title='Longitudinal Phasespace Distribution', 
                xlabel=r'$\Delta z$ [$\mu$m]', ylabel=r"$\delta$", 
                nbins=100, projection=7, density=False, cmap='viridis', figname=None)

### Save beam distribution for JuTrack

In [ ]:
beam_io_manager.write('jutrack', filename='jubeam.dat', beam=beam)

In [ ]:
beam_metadata = beam_io_manager.read_beam_metadata('jubeam.dat')

## Julia Integration

In [ ]:
# PATH Environment Setting for julia
import os
current_path = os.environ.get('PATH', '')
pyenv_bin_path = os.path.expanduser("~/.pyenv/bin")
julia_bin_path = os.path.expanduser("~/Work/simulation_codes-working/julia/usr/bin")
new_path_dirs = [pyenv_bin_path, julia_bin_path]
if current_path:
    new_path_dirs.extend(current_path.split(os.pathsep))
new_path = os.pathsep.join(filter(None, new_path_dirs))
os.environ['PATH'] = new_path

# the number of threads for julia computing
%env JULIA_NUM_THREADS = 2

In [ ]:
from julia import Main

Main.eval("using Pkg")
Main.eval('Pkg.activate("/home/cspark/Work/simulation_codes-working/JuTrack.jl")')
Main.eval("Pkg.instantiate()")

Main.eval("using JuTrack")

### Create Inital Beam Distribution for JuTrack

In [ ]:
jubeam = Main.eval('include("beam.jl")')

In [ ]:
julattice = Main.eval('include("lebt.jl")')

### Initial Beam Distribution

In [ ]:
beam_state0 = jubeam.r
x = beam_state0[:,0]
xp = beam_state0[:,1]
y = beam_state0[:,2]
yp = beam_state0[:,3]
z = beam_state0[:,4]
delta = beam_state0[:,5]

In [ ]:
mass_number = beam.mass_number
charge_state = beam.charge_state
beam_current = beam.beam_current
reference_energy = beam.reference_energy
print (mass_number, charge_state, beam_current, reference_energy)

In [ ]:
jubeam_conv = JuTrackIO.convert(beam_state0, mass_number, charge_state, beam_current, reference_energy)
jubeam_conv.state

In [ ]:
jubeam_conv.centroid

In [ ]:
jubeam_twiss = Twiss(jubeam_conv)
print(twiss.values()['twiss_x']['emittance'])
print(twiss.values()['twiss_y']['emittance'])
print(twiss.values()['twiss_z']['emittance'])

In [ ]:
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
y0 = jubeam_conv.centroid['y']
yp0 = jubeam_conv.centroid['yp']

In [ ]:
print("initial beam emittance using JuTrack")
Main.eval('get_emittance!(beam::Beam)')
Main.eval('println(beam.emittance)')

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -20, 20], title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
phasespace_plot(y*1e3, yp*1e3, x_center=y0, y_center=yp0, xyrange=[-20, 20, -20, 20], title='Vertical Phasespace Distribution', 
                xlabel=r'$y$ [mm]', ylabel=r"$p_y / p_z$ [mrad]",
                nbins=100, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
phasespace_plot(z*1e-6, delta*1e-2, xyrange=[-20, 20, -10e-6, 10e-6], title='Longitudinal Phasespace Distribution', 
                xlabel=r'$\Delta z$ [$\mu$m]', ylabel=r"$\delta$", 
                nbins=100, projection=7, density=True, cmap='viridis', figname=None)

### After LEBT Beam Distribution

In [ ]:
Main.eval('linepass!(LEBT, beam)')
print("beam emittance at the LEBT exit using Jutrack")
Main.eval('get_emittance!(beam::Beam)')
Main.eval('println(beam.emittance)')

In [ ]:
beam_state = Main.beam.r
x = beam_state[:,0]
xp = beam_state[:,1]
y = beam_state[:,2]
yp = beam_state[:,3]
z = beam_state[:,4]
delta = beam_state[:,5]

In [ ]:
jubeam_conv = JuTrackIO.convert(beam_state, mass_number, charge_state, beam_current, reference_energy)
jubeam_conv.state

In [ ]:
jubeam_conv.centroid

In [ ]:
jubeam_twiss = Twiss(jubeam_conv)
print(jubeam_twiss.values()['twiss_x']['emittance'])
print(jubeam_twiss.values()['twiss_y']['emittance'])
print(jubeam_twiss.values()['twiss_z']['emittance'])

In [ ]:
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
y0 = jubeam_conv.centroid['y']
yp0 = jubeam_conv.centroid['yp']

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
phasespace_plot(y*1e3, yp*1e3, x_center=y0, y_center=yp0, xyrange=[-20, 20, -40, 40], title='Vertical Phasespace Distribution', 
                xlabel=r'$y$ [mm]', ylabel=r"$p_y / p_z$ [mrad]",
                nbins=100, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
phasespace_plot(z*1e-6, delta*1e-2, xyrange=[-20, 20, -10e-6, 10e-6], title='Longitudinal Phasespace Distribution', 
                xlabel=r'$\Delta z$ [$\mu$m]', ylabel=r"$\delta$", 
                nbins=100, projection=7, density=True, cmap='viridis', figname=None)

### Tracking

In [ ]:
jubeam = Main.eval('include("beam.jl")')

In [ ]:
track = Main.eval('include("track.jl")')

In [ ]:
states_at_diags = Main.states_at_diags

In [ ]:
states_at_diags

In [ ]:
x = states_at_diags["WS01"][:,0]
xp = states_at_diags["WS01"][:,1]
y = states_at_diags["WS01"][:,2]
yp = states_at_diags["WS01"][:,3]
z = states_at_diags["WS01"][:,4]
delta = states_at_diags["WS01"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS01"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["AS01"][:,0]
xp = states_at_diags["AS01"][:,1]
y = states_at_diags["AS01"][:,2]
yp = states_at_diags["AS01"][:,3]
z = states_at_diags["AS01"][:,4]
delta = states_at_diags["AS01"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["AS01"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["WS02"][:,0]
xp = states_at_diags["WS02"][:,1]
y = states_at_diags["WS02"][:,2]
yp = states_at_diags["WS02"][:,3]
z = states_at_diags["WS02"][:,4]
delta = states_at_diags["WS02"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS02"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["WS03"][:,0]
xp = states_at_diags["WS03"][:,1]
y = states_at_diags["WS03"][:,2]
yp = states_at_diags["WS03"][:,3]
z = states_at_diags["WS03"][:,4]
delta = states_at_diags["WS03"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS03"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["WS04"][:,0]
xp = states_at_diags["WS04"][:,1]
y = states_at_diags["WS04"][:,2]
yp = states_at_diags["WS04"][:,3]
z = states_at_diags["WS04"][:,4]
delta = states_at_diags["WS04"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS04"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["D23"][:,0]
xp = states_at_diags["D23"][:,1]
y = states_at_diags["D23"][:,2]
yp = states_at_diags["D23"][:,3]
z = states_at_diags["D23"][:,4]
delta = states_at_diags["D23"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["D23"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

### Tracking with Space Charge Effects

In [ ]:
jubeam = Main.eval('include("./beam.jl")')
julattice = Main.eval('include("./lebt_sc.jl")')

In [ ]:
track = Main.eval('include("./track_sc.jl")')

In [ ]:
states_at_diags = Main.states_at_diags

In [ ]:
x = states_at_diags["WS01"][:,0]
xp = states_at_diags["WS01"][:,1]
y = states_at_diags["WS01"][:,2]
yp = states_at_diags["WS01"][:,3]
z = states_at_diags["WS01"][:,4]
delta = states_at_diags["WS01"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS01"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["AS01"][:,0]
xp = states_at_diags["AS01"][:,1]
y = states_at_diags["AS01"][:,2]
yp = states_at_diags["AS01"][:,3]
z = states_at_diags["AS01"][:,4]
delta = states_at_diags["AS01"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["AS01"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["WS02"][:,0]
xp = states_at_diags["WS02"][:,1]
y = states_at_diags["WS02"][:,2]
yp = states_at_diags["WS02"][:,3]
z = states_at_diags["WS02"][:,4]
delta = states_at_diags["WS02"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS02"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["WS03"][:,0]
xp = states_at_diags["WS03"][:,1]
y = states_at_diags["WS03"][:,2]
yp = states_at_diags["WS03"][:,3]
z = states_at_diags["WS03"][:,4]
delta = states_at_diags["WS03"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS03"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["WS04"][:,0]
xp = states_at_diags["WS04"][:,1]
y = states_at_diags["WS04"][:,2]
yp = states_at_diags["WS04"][:,3]
z = states_at_diags["WS04"][:,4]
delta = states_at_diags["WS04"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["WS04"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)

In [ ]:
x = states_at_diags["D23SC"][:,0]
xp = states_at_diags["D23SC"][:,1]
y = states_at_diags["D23SC"][:,2]
yp = states_at_diags["D23SC"][:,3]
z = states_at_diags["D23SC"][:,4]
delta = states_at_diags["D23SC"][:,5]

jubeam_conv = JuTrackIO.convert(states_at_diags["D23SC"], mass_number, charge_state, beam_current, reference_energy)
x0 = jubeam_conv.centroid['x']
xp0 = jubeam_conv.centroid['xp']
jubeam_conv.centroid

In [ ]:
phasespace_plot(x*1e3, xp*1e3, x_center=x0, y_center=xp0, xyrange=[-20, 20, -40, 40], 
                title='Horizontal Phasespace Distribution', 
                xlabel=r'$x$ [mm]', ylabel=r"$p_x / p_z$ [mrad]",
                nbins=200, projection=0, density=False, cmap='viridis', figname=None)